In [1]:
import Pkg; Pkg.activate(@__DIR__)
using RobotZoo
using RobotDynamics
using LinearAlgebra
using StaticArrays
using SparseArrays
using MeshCat, GeometryBasics, Colors, CoordinateTransformations, Rotations
using Plots
using BilinearControl
using BilinearControl.EDMD
using FileIO, JLD2

  Activating environment at `~/.julia/dev/BilinearControl/examples/Project.toml`


In [3]:
ref_traj = FileIO.load("../data/cartpole_eDMD_data.jld2")
A = ref_traj["F"]
C = ref_traj["C"]
g = ref_traj["g"]
Z_sim = ref_traj["Z_sim"]

601-element Vector{Vector{Float64}}:
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
 [1.0, 4.4363552683616357e-5, -8.87234785907435e-5, 0.017745289179026392, -0.03548767692522162, 4.436355266906419e-5, -8.872347847434043e-5, 0.017744357875698412, -0.035480228677459674, 0.9999999990159376, 0.9999999960640722, 0.9998425564875598, 0.9993703784748651]
 [1.0, 0.0001743224680746385, -0.0003485871659918609, 0.034237890560250464, -0.06845277619248052, 0.00017432246719174395, -0.00034858715893221486, 0.03423120182058373, -0.06839932957840379, 0.9999999848058386, 0.9999999392434945, 0.9994139406781949, 0.9976580234294841]
 [1.0, 0.0003816339671567053, -0.0007629787102985141, 0.04868609253081929, -0.09729583225046808, 0.0003816339578928916, -0.0007629786362722221, 0.048666861080437225, -0.09714239673481612, 0.9999999271777584, 0.9999997089317579, 0.9988150662823311, 0.9952704932613121]
 [1.0, 0.0006576079247211254, -0.001314315289108891, 0.06170266962409919, -0.123228141033466

In [4]:
const RD = RobotDynamics

RobotDynamics

In [5]:
function defcolor(c1, c2, c1def, c2def)
    if !isnothing(c1) && isnothing(c2)
        c2 = c1
    else
        c1 = isnothing(c1) ? c1def : c1
        c2 = isnothing(c2) ? c2def : c2
    end
    c1,c2
end

function set_mesh!(vis0, model::RobotZoo.Cartpole; 
        color=nothing, color2=nothing)
    vis = vis0["robot"]
    dim = Vec(0.1, 0.3, 0.1)
    rod = Cylinder(Point3f0(0,-10,0), Point3f0(0,10,0), 0.01f0)
    cart = Rect3D(-dim/2, dim)
    hinge = Cylinder(Point3f0(-dim[1]/2,0,dim[3]/2), Point3f0(dim[1],0,dim[3]/2), 0.03f0)
    c1,c2 = defcolor(color,color2, colorant"blue", colorant"red")

    pole = Cylinder(Point3f0(0,0,0),Point3f0(0,0,model.l),0.01f0)
    mass = HyperSphere(Point3f0(0,0,model.l), 0.05f0)
    setobject!(vis["rod"], rod, MeshPhongMaterial(color=colorant"grey"))
    setobject!(vis["cart","box"],   cart, MeshPhongMaterial(color=isnothing(color) ? colorant"green" : color))
    setobject!(vis["cart","hinge"], hinge, MeshPhongMaterial(color=colorant"black"))
    setobject!(vis["cart","pole","geom","cyl"], pole, MeshPhongMaterial(color=c1))
    setobject!(vis["cart","pole","geom","mass"], mass, MeshPhongMaterial(color=c2))
    settransform!(vis["cart","pole"], Translation(0.75*dim[1],0,dim[3]/2))
end

function visualize!(vis, model::RobotZoo.Cartpole, x::StaticVector)
    y = x[1]
    θ = x[2]
    q = expm((pi-θ) * @SVector [1,0,0])
    settransform!(vis["robot","cart"], Translation(0,-y,0))
    settransform!(vis["robot","cart","pole","geom"], LinearMap(UnitQuaternion(q)))
end

function visualize!(vis, model::RobotDynamics.AbstractModel, tf::Real, X)
    fps = Int(round((length(X)-1)/tf))
    anim = MeshCat.Animation(fps)
    n = state_dim(model)
    for (k,x) in enumerate(X)
        atframe(anim, k) do
            x = X[k]
            visualize!(vis, model, SVector{n}(x)) 
        end
    end
    setanimation!(vis, anim)
end

function simulate(dmodel, x0, U, dt)

    t = 0.0
    x = x0
    X = [x]
    T = [t]
    
    for k in 1:length(U)

        u = U[k]
        x = RD.discrete_dynamics(dmodel, x, u, t, dt)
        t += dt
        push!(X, x)
        append!(T, t)

    end

    return T, X

end

simulate (generic function with 1 method)

In [6]:
struct BilinearCartpole <: RD.DiscreteDynamics
    A::Matrix{Float64}
    C::Matrix{Float64}
    g::Matrix{Float64}
end

RD.state_dim(::BilinearCartpole) = 13
RD.control_dim(::BilinearCartpole) = 1

function discrete_dynamics(model::BilinearCartpole, zk, uk, t, dt)
    
    zn = model.A * zk + (model.C * zk) .* uk
    return zn
    
end

function discrete_dynamics!(model::BilinearCartpole, zn, uk, t, dt)
    
    zn .= model.A * zn + (model.C * zn) .* uk
    
end

function simulate_bilinear(model::BilinearCartpole, x0, z0, U, T)
    
    dt = T[2] - T[1]
    
    x = x0
    z = z0
    Z = [z]
    X = [x]

    for k in 1:length(U)

        u = U[k]
        t = T[k]
        
        z = discrete_dynamics(model, z, u, t, dt)
        x = g * z

        push!(Z, z)
        push!(X, x)
        
    end

    return X, Z

end

simulate_bilinear (generic function with 1 method)

In [7]:
dmodel = BilinearCartpole(A, C, g)
n, m = RD.dims(dmodel)

(13, 1, 13)